In [ ]:
!pip install transformers huggingface_hub 
!pip install "git+https://github.com/Tfloow/auto_adpq.git"

In [ ]:
hugging_face-cli login

In [ ]:
# Load llama 3.1 8B model and quantize it with Auto-AdpQ
from transformers import AutoModelForCausalLM, AutoTokenizer
from auto_adpq import Auto_AdpQ, AutoAdpQConfig

model_name = "meta-llama/Llama-3.1-8B"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="cpu", torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)